In [21]:
import os
import pickle
import pandas as pd

files = os.listdir('speech2text/created_data')
q = len(files)
df = pd.DataFrame(columns=['input_values', 'labels'], index=range(q))
i = 0
for file in files:
    path = 'speech2text/created_data/' + file
    with open(path, 'rb') as fp:
        itemlist = pickle.load(fp)

    df['input_values'][i] = itemlist[0]['input_values']
    df['labels'][i] = itemlist[0]['labels']
    i+=1

df.head()

,input_values,labels
0,"[[tensor(-1.5662e-05), tensor(-1.5662e-05), te...","[[tensor(7), tensor(24), tensor(5), tensor(6),..."
1,"[[tensor(-0.0006), tensor(-0.0006), tensor(-0....","[[tensor(7), tensor(24), tensor(5), tensor(6),..."
2,"[[tensor(-4.8199e-05), tensor(-4.8199e-05), te...","[[tensor(7), tensor(19), tensor(7), tensor(13)..."
3,"[[tensor(-0.0002), tensor(-0.0002), tensor(-0....","[[tensor(7), tensor(14), tensor(5), tensor(9),..."
4,"[[tensor(0.0003), tensor(0.0003), tensor(0.000...","[[tensor(24), tensor(16), tensor(15), tensor(1..."


In [22]:
df.shape

(13735, 2)

In [9]:
from datasets import load_metric
import pickle
path = "speech2text/pharmacy6.pickle"
with open(path, 'rb') as output:
  data = pickle.load(output)

batch = list(data.keys())
letters = ''
for word in batch:
    for i in word.upper().split():
        letters += i
print(letters)

ABETALIPOPROTEINEMIAABRUPTIOPLACENTAEACARBOSEACETAMINOPHENACETOHEXAMIDEACETYLSALICYLICACIDACHALASIAACHONDROPLASIAACOUSTICNEUROMAACROCYANOSISACROMEGALYACTINICCHEILOSISACTINOMYCOSISACUTEINTERMITTENTPORPHYRIAACUTELYMPHOCYTICLEUKEMIAACUTEMYELOIDLEUKEMIAACUTENEPHRITICSYNDROMEACYCLOVIRADENOCARCINOMAADENOMAADENOSINEADRENALINEADRENOLEUKODYSTROPHYADRENOMYELONEUROPATHYADYNAMICILEUSAEROPHAGIAAEROTITISAGAMMAGLOBULINEMIAAGEUSIAAGNOGENICMYELOIDMETAPLASIAAGNOSIAAGORAPHOBIAAGRAPHIAAKATHISIAALANINETRANSAMINASEALBUTEROLALDOSTERONEALEXIAALKALOSISALKYLATINGAGENTSALLERGICBRONCHOPULMONARYASPERGILLOSISALLOPURINOLALOPECIAALPHA-FETOPROTEINALPHA-THALASSEMIAALPHA-TOCOPHEROLALVEOLARCELLCARCINOMAALVEOLIAMANTADINEAMBLYOPIAAMEBIASISAMEBOMAAMENORRHEAAMINOCAPROICACIDAMINOGLUTETHIMIDEAMINOGLYCOSIDESAMITRIPTYLINEAMNIOCENTESISAMNIOTICFLUIDAMPHETAMINESAMPHOTERICINBAMPULLAOFVATERAMYLOIDOSISAMYOTROPHICLATERALSCLEROSISANAEROBICANAPHYLACTOIDANAPHYLACTOIDREACTIONSANAPHYLAXISANCYLOSTOMADUODENALEANENCEPHALYANESTHESIAANEURYSMANGI

In [31]:
import re
chars_to_ignore_regex = '[\,\?\.\!\;\:\(\)\#]'
letters = re.sub(chars_to_ignore_regex, '', letters).upper()
print(letters)

...


In [32]:
vocab_dict = {}
count = 0
for i,let in enumerate(letters):
    if not(let in vocab_dict.keys()):
        vocab_dict[let]= count
        count +=1
print(vocab_dict)


{'A': 0, 'B': 1, 'E': 2, 'T': 3, 'L': 4, 'I': 5, 'P': 6, 'O': 7, 'R': 8, 'N': 9, 'M': 10, 'U': 11, 'C': 12, 'S': 13, 'H': 14, 'X': 15, 'D': 16, 'Y': 17, 'G': 18, 'K': 19, 'V': 20, 'F': 21, 'Z': 22, 'J': 23, 'Q': 24, 'W': 25, '6': 26, '3': 27, '4': 28, '2': 29, '5': 30, '0': 31, '1': 32}


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [4]:
vocabs = batch.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True)

AttributeError: 'list' object has no attribute 'map'

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
###ensemble preprocess
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, HubertForCTC
from torch.optim.lr_scheduler import StepLR
import torch
import librosa
import pickle
import pydub
import numpy as np


class prepare_data():

    def __init__(self):
        self.processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
        self.model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")


    def read(f, normalized=False):
        """MP3 to numpy array"""
        a = pydub.AudioSegment.from_mp3(f)
        a = a.set_frame_rate(16000)
        y = np.array(a.get_array_of_samples())
        if a.channels == 2:
            y = y.reshape((-1, 2))
        if normalized:
            return a.frame_rate, np.float32(y) / 2**15
        else:
            return a.frame_rate, y


    def prep_data(self, path):

        with open(path, 'rb') as output:
          data = pickle.load(output)

        l = 0
        x_train = []
        y_train = []
        bug = {}
        # load audio and train
        for drug in data.keys():

            l += 1
            for j in range(len(data[drug]['brand_audio'])):

                file = data[drug]['brand_audio'][j]['path']
                path = 'speech2text' + file[1:]

                if path[-3:] == 'wav':

                  try:
                      speech, _ = librosa.load(path, sr=16000)

                  except:
                      bug[drug] = j
                      continue

                elif path[-3:] == 'mp3':

                  try:
                      _, speech = self.read(path, normalized=True)
                  except:
                      bug[drug] = j
                      continue

                else:
                    bug[drug] = j
                    continue

                input_values = self.processor(speech, sampling_rate=_, return_tensors="pt")
                target_transcription = drug.upper()

            # encode labels
                with self.processor.as_target_processor():
                  input_values['labels'] = self.processor(target_transcription, return_tensors="pt").input_ids

                x_train.append(input_values)

        return x_train, bug

2769
